In [39]:
import os 
import pandas as pd
from tqdm import tqdm
from src.dysregnet.linearmodel import LinearModel
import statsmodels.api as sm
import numpy as np

from tqdm import tqdm
import numpy as np
from scipy import stats
import statsmodels.api as sm
import pickle
from scipy.stats import combine_pvalues
#_____________

In [41]:
with open('lll.pickle', 'rb') as handle:
    lll = pickle.load(handle)
lll

[]

In [ ]:
with open('blb.pickle', 'rb') as handle:
    blb = pickle.load(handle)
blb

In [ ]:
with open('lll.pickle', 'rb') as handle:
    lll = pickle.load(handle)
lll

In [ ]:
with open('lbl.pickle', 'rb') as handle:
    lbl = pickle.load(handle)
lbl

In [33]:
os.getcwd()
current_dir = os.getcwd()
grn_breast = pd.read_csv(current_dir + "/GENIE3_output/linkedList_output_slurm_gene_tpm_v10_breast_mammary_tissu_filtered.csv")
grn_lung = pd.read_csv(current_dir + "/GENIE3_output/linkedList_output_slurm_gene_tpm_v10_lung_filtered.csv")

exp_lung = pd.read_csv(current_dir + "/gtex/gene_tpm_v10_lung_filtered.csv")
exp_breast = pd.read_csv(current_dir + "/gtex/gene_tpm_v10_breast_mammary_tissu_filtered.csv")
 

In [34]:
exp_lung = exp_lung.set_index(exp_lung.columns[0])
exp_lung = exp_lung.T 
#exp_lung.insert(0, "sample", exp_lung. index)
#assert all(exp_lung.iloc[:, 0].values == meta_lung.iloc[:,0].values)
exp_lung.head(1)

meta_lung = pd.DataFrame({
    'sample': exp_lung.columns[1:],
    'condition': np.random.choice([0], size=len(exp_lung.columns[1:])),
    'gender' : 'FEMALE'
})
meta_lung

,sample,condition,gender
0,OR4F5,0,FEMALE
1,CICP27,0,FEMALE
2,WASH9P,0,FEMALE
3,MTND1P23,0,FEMALE
4,MTND2P28,0,FEMALE
...,...,...,...
21036,MT-ND4,0,FEMALE
21037,MT-ND5,0,FEMALE
21038,MT-ND6,0,FEMALE
21039,MT-TE,0,FEMALE


In [35]:
exp_breast = exp_breast.set_index(exp_breast.columns[0])
exp_breast = exp_breast.T 
exp_breast.insert(0, "sample", exp_breast. index)
#assert all(exp_breast.iloc[:, 0].values == meta_breast.iloc[:,0].values)
exp_breast.head(3)

grn_breast
meta_breast = pd.DataFrame({
    'sample': exp_breast.columns[1:],
    'condition': np.random.choice([0], size=len(exp_breast.columns[1:])),
    'gender' : 'FEMALE'
})
meta_breast.head(1)

,sample,condition,gender
0,WASH7P,0,FEMALE


In [36]:
grn = grn_breast
expr = exp_breast
tissue = "breast"
model_dir = current_dir + "/models/" + tissue
meta = meta_breast

In [37]:
found = 0
skipped = 0
notfound = 0
notskipped = 0

conCol='condition'
CatCov=['gender'] 
control = meta[ meta[conCol]==0 ].index.values.tolist()
        
for tup in tqdm(grn.itertuples(), desc="Processing edges"):
            edge = (tup[1], tup[2])  # Extract TF → target pair 

            # Skip self-loops
            if edge[0] == edge[1]:
                continue

            # print("Loading model for edge: ", edge)
            filename = os.path.join(model_dir, f"{edge[0]}_{edge[1]}.pkl")
            # print("Filename: ", filename)
            try:
                #with open(filename, "rb") as file:
                    # Load pre-trained model
                    # results = pickle.load(file)
                results = LinearModel.load(filename)
                found +=1
            except FileNotFoundError:
                print(f"Model file not found for edge {edge}. Skipping.")
                notfound +=1
                continue

            # check if no control samples >3
            control=expr.loc[control]
            #case=EXPR.expr.loc[data.case]  
            # check if trained models fit control data, use only these and ignore others
            # print('Only 3 control samples, using only these for the model')
            # prepare control for fitting model TODO correct?
            
            x_train = control[  [edge[0]] ]#+ covariate_name ]
            x_train = sm.add_constant(x_train, has_constant='add') # add bias
            y_train = control[edge[1]].values

            residuals = y_train - results.predict(x_train)
            mean_residual = np.mean(residuals)
            std_residual = np.std(residuals)
            z_scores = (residuals - mean_residual) / std_residual

            pvalues = stats.norm.sf(abs(z_scores))
            combined_pvalue, _ = combine_pvalues(pvalues, method='fisher')

            # Identify significant deviations
            significant = np.abs(z_scores) > 2
            # skip model if too many significant deviations
            alpha = 0.05
            if combined_pvalue < alpha:
                #print("Warning: Too many significant deviations. Skipping model.")
                skipped +=1
                continue
            notskipped += 1
        

Processing edges: 0it [00:00, ?it/s]


KeyError: "None of [Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,\n       ...\n       20387, 20388, 20389, 20390, 20391, 20392, 20393, 20394, 20395, 20396],\n      dtype='int64', length=20397)] are in the [index]"